En este notebook aprenderemos a trabajar con variables aleatorias, de tal forma que podamos hacer simulaciones de sistemas estocásticos. Aprenderemos entre otras cosas el método de Monte Carlo para calcular integrales y también esbozaremos el método de Montecarlo para hacer simulaciones de partículas bajo un potencial determinado. 

# Variables aleatorias uniformes

Lo primero que necesitas es poder responder ¿qué es una variable aleatoria? 

Definición de wikipedia: 

> Una variable aleatoria es una función que asigna un valor, usualmente numérico, al resultado de un experimento aleatorio

Matemáticamente una variable aleatoria es una función que va de un espacio de muestreo a los reales (generalmente, aunque puede ir a cualquier conjunto numérico). Entonces, un variable aleatoria no es realmennte una variable y tampoco es aleatoria, lo que es aleatorio es el experimento para elegir un elemento del espacio de muestreo. 

En general, una variable aleatoria, puede tener cualquier tipo de distribución, es decir, si se hacen suficientes (infinitos) experimentos y se produce un histograma de los valores que se obtengan, el histograma podría seguir cualquier tipo de función. Lo más común es querer tener una distribución uniforme en un intervalo, concretamente, en el intervalo $(0,1)$. 

En Julia, esta clase de variables aleatorias, con distribución uniforme sobre el intervalo $(0,1)$ se producen utilizando la función rand(). Si te interesa saber precisamente cómo genera estos números, puedes leer el siguiente artículo: http://www.math.sci.hiroshima-u.ac.jp/~m-mat/MT/ARTICLES/dSFMT.pdf

Una forma de pasar del intervalo $(0,1)$ a cualquier otro intervalo es mediante una transformación lineal. Primero agrandas el intervalo al tamaño deseado, multiplicando la variable aleatoria por un valor determinado $a*rand()$, y después desplazas el intervalo al punto deseado, $a*rand() + b$. Por ejemplo, si se quisera una variable aleatoria con distribución uniforme en el intervalo $(-3,5)$, lo primero que tendríamos que hacer, es medir el tamaño del nuevo intervalo, que en este caso es 8. Eso implica que $a = 8$, después tendríamos que ver qué el desplazamiento de nuestro intervalo y eso es, cuántas unidades está recorrido el primer valor del intervalo, esto es $b = -3$, por lo tanto, nuestra variable aleatoria sería simplemente $8*rand()-3$.

[1] Produce un número aleatorio con distribución uniforme en el intervalo $(-25,4)$. 

Ok, ya sabemos producir variables aleatorias en cualquier intervalo. ¿Son correctas? Sí lo son, pero con un detalle, el número de posibilidades se restringe al número de posibilidades que hay en el intervalo $(0,1)$, lo cual no necesariamente es lo que quisieramos. En caso de requerir muchas más posibilidades, tendríamos que hacer nuestro propio generador de números aleatorios siguiendo los pasos del artículo que puse un poco más arriba. Sin embargo, en general es suficiente. 

¿Cómo verificar que nuestra variable aleatoria es realmente uniforme? Para esto podemos hacer un histograma. En algunos casos es necesario hacerlos a mano, es decir, hacer una función que cuente cuántos valores caen en cada sub intervalo de tamaño $a/n$, con $n$ el número de subintervalos. Graficando esa función contra el valor del centro del intervalo, obtendríamos el histograma correspondiente. El histograma de una variable aleatoria uniforme en un intervalo dado, debería parecerse más y más a la gráfica de una constante en ese intervalo al incrementar el número de valores aleatorios del histograma. 

Para hacer estos histogramas, podemos usar una función de Plots llamada histogram. En esta función basta con dar el arreglo de valores aleatorios y el número de intervalos que tendrá el histograma para producir dicho histograma. Por ejemplo, digamos que yo tengo un arreglo de números aleatorios X y quiero hacer un histograma con 10 subintervalos, entonces, bastaría con escribir: histogram(X, nbins=10). 

Falta mencionar, cómo generar un arreglo con n números aleatorios. Uno puede hacer un arreglo vacío y luego un ciclo for, donde empuje los valores de aleatorios dentro del arreglo vacío, pero de hecho Julia ya incluye una función para hacer esto y es precisamente rand, sólo que en vez de dejar el paréntesis vacío, hay que poner n, el número de valores aleatorios que tendrá nuestro arreglo. Es decir rand(n). 

[2] Haz un histograma de $10^3$, $10^4$ y $10^5$ valores, con $100$ subintervalos, sobre la variable aleatoria que generaste, en el intervalo $(-25,4)$.  

# Variables Aleatorias no uniformes

¿Qué haremos si en una de nuestros modeos requerimos una función que tenga una variable aleatoria con una distribución específica? 

Si requerimos una distribución gaussiana (que es lo más común), podemos usar la función randn(n), que nos arrojará un arreglo de n números aleatorios con la distribución normal. Trasladar los valores de la gausiana o estirar la gausiana sigue un proceso muy similar a lo que se explocó para una variable aleatoria uniforme. 

[3] Haz el histograma de un arreglo de 10^5 valores aleatorios con una distribución normal. 

Pero más difícil aún, es si requerimos distribuciones particulares y extrañas. Para lograr conseguir cualquier distribución $f$, lo que necesitamos es primero obtener cual es la función cumulativa de esa distribución, es decir, la integral desde $- \infty$ hasta $x$ de $f$. Entonces, si esta función es $F(x) = \int_{-\infty}^x$, $F^{-1}(rand())$ tendrá la distribución $f$.  

Esto no es siempre fácil, pero podemos hacerlo para algunos casos directos. Por ejemplo, para la distribución exponencial $f(x) = \lambda exp(\lambda x) $ si $x>0$ y $0$ si $x<0$, su cumulativa es $F(x) = 1- exp(-\lambda x)$, por lo tanto, $F^{-1}(x) = \frac{-1}{\lambda}log(1-x)$. 

[4] Haz el histograma de un arreglo de 10^5 elementos con distribución exponencial. 

# Monte Carlo para calcular áreas. 

Ok, ya tenemos una forma de generar números aleatorios que es descente. Ahora hay que pensar cómo usar esto. A veces necesitamos calcular áreas de regiones complicadas de las cuales es incluso difícil escribir adecuadamente las funciones que las describen. Por ejemplo, supongamos que quisiéramos calcular el área que encierran decenas de círculos puestos aleatoriamente en un cuadrado. El área de un círculo es simple de calcularse, pero cuando se trata de círculos encimados, el problema se complica. Más aún, si lo que se encima no son círculos, sino curvas más o menos complicadas. ¿Qué podemos hacer en este caso? Una forma de calcular el área es aproximarla por un cuadrado que encierre a nuestro objeto y calcular el área del cuadrado. Si ninguna parte del objeto se sale del cuadrado y el objeto intersecciona al cuadrado en sus cuatro lados, la aproximación no será tan mala, aunque aún, si nuestro objeto es "delgado", el error aún será grande. Ahora podemos lanzar "dardos" aleatorios dentro de nuestro cuadrado. Esto es, basta con generar números aleatorios con distribución uniforme sobre un intervalo para las coordenadas $x$ y otro para las coordenadas $y$. Después contar cuantos "dardos" cayeron dentro del objeto y cuantos lanzamos en total. La probabilidad de atinarle al objeto será proporcional al área del objeto e inversamente proporcional al área del cuadrado. Es decir $p = A_o / A_c$ donde $p$ es la probabilidad de atinarle al objeto, $A_o$ es el área del objeto y $A_c$ es el área del cuadrado. 

La interpretación frecuentista de la probabilidad nos dice que $p \sim N_o /N_t$ donde $N_o$ es el número de dardos que cayeron dentro del objeto y $N_t$ es el número de dardos que se lanzaron en total. 

[5] Has una simulación que lance "dardos" aleatorios en un cuadrado de lado 2, centrado en el origen. En tu simulación cuenta cuantos dardos cayeron dentro de un círculo de radio $1$ y que con ello calcula el área de tu círculo. ¿Coincide con lo esperado? 

Este método es aceptablemente bueno, pero como habrás visto, no es perfecto; sin embargo, en muchos cálculos es la forma más rápida de hacer un cálculo rápido, si no se requiere tanta precisión. 

# Monte-Carlo para calcular integrales

Quizá habrás escuchado que el algoritmo de Monte-Carlo fue el que hizo a los estadounidenses ganar la segunda guerra mundial, pues con él pudieron hacer rápidamente los cálculos necesarios para la fabricación de la bomba atómica. Este algoritmo no se refiere al que arriba desarrollaste, sino a uno para hacer cálculos de integrales rápidas. 

Supón que quieres evaluar $\int_a^b g(x)dx$ y asume que no existe una fórmula simple para hacer esta evaluación. ¿Cómo lo harías? Quizá una integral por sumas de Riemman, pero en muchos casos, estas suman pueden parecer que divergen u obtener resultados lejanos al resultado real dado el error numérico (como ya quizá comprobaste). Entonces, lo mejor es una integración por el método de Monte-Carlo que veremos a continuación. 

Algoritmo de Monte-Carlo para el intervalo $(0,1)$ es el siguiente: 

- Genera n valores aleatorios $x_i$ en el intervalo $(0,1)$.  
- aplica la función $g$ a cada uno de estos invervalos $g(x_i) = g_i$
- Haz el promedio de los $g_i$, es decir $\sum g_i /n$

Ese valor será tu integral. 

Entonces, para cualquier intervalo, el algoritmo sería el siguiente: 

- Genera n valores aleatorios $x_i$ en el intervalo $(a,b)$.  
- aplica la función $g$ a cada uno de estos invervalos $g(x_i) = g_i$
- Haz el promedio de los $g_i$, es decir $\sum g_i /n$
- Multiplica este valor por el tamaño del intervalo $(b-a)$

ese será el valor de la integral. 

[6] Haz una función que integre por el método de Monte-Carlo, dada una función $g$, un intervalo $(a,b)$ y un número de puntos aleatoreos generados $n$. 

[7] prueba tu función integral con diferentes funciones. Intenta funciones complicadas, donde la regla de simpson falle y tambiél la integración por sumas de Riemann. Aunque asegúrate que tu función sí sea integrable (no diverja).

[8] Mide el error de este método de integración comparado con el método de sumas de Riemann. O sea, calcula la solución exacta y ve cuanto falla el método de Monte-Carlo y cuanto falla el de sumas de Riemann. 

# Monte Carlo para simulaciones termodinámicas

Ya habrás notado, que aunque hay una relación entre el método para calcular áreas y el método para calcular integrales, no es directo y sin embargo se llaman ambos métodos igual. De forma se llaman métodos de Monte-Carlo, a todos los métodos que utilicen la generación de números aleatorios para hacer algún cálculo. El nombre está inspirado en el gran casino de Monte-Carlo, donde la gente va a apostar grandes sumas de dinero y de alguna forma es uno de los más grande laboratorios de probabilidad. 

En el caso de las simulaciones dinámicas, el algoritmo más conocido es el algoritmo de Metrópolis. Por lo tanto, se dice que las simulaciones son de tipo Monte-Carlo siguiendo el algoritmo de Metrópolis, de forma simplificada a veces se refiere sólo a simulaciones de Monte-Carlo o a Simulaciones de Metrópolis, lo cual genera, por supuesto, confusiones. 

La idea básica de este algoritmo es la siguiente: 

Se elige un elemento de nuestro espacio de muestreo de forma aleatoria. Se aplica una función dada (por ejemplo se mueve una partícula, se cambia alguna propiedad, etc...) se revisa si en la nueva configuración se está "mejor" que antes (por ejemplo, si la energía se redujo o se incrementó), si se mejoró la situación, entonces se mantiene la nueva configuración, sino, se da una probabilidad de que se rechace la configuración y esta probabilidad depente de qué tan "mal" estamos en la nueva configuración.  Se tira un número aleatorio entre 0 y 1 y si este número es mayor que la probabilidad, se rechaza la nueva configuración, si es menor, se acepta a pesar de ser una mala configuración. 

Por supuesto, parte del problema aquí es definir cual es la buena probabilidad de aceptar una "mal" situación. Para esto nos basamos en los resultados de la física estadística (que sé que aún no tienen). La probabilidad de tener un estado dado es $$P(\sigma) = exp(-H(\sigma)/(KT))/Z$$ donde $\sigma$ es el estado, $H$ la energía del sistem, $K$ la constante de Boltzman, $T$ la temperatura y $Z$ es la función de partición (que es una constante). 

Entonces, la probabilidad de pasar de un estado a otro, será $$p = P(\sigma'|\sigma) = min(1, e^{-H(\sigma')/(KT)}/e^{-H(\sigma)/(KT)})$$

Así, lo único que necesitamos para calcular esta probabilidad es el hamiltoniano $H$. El algoritmo es entonces el siguiente: 

1. Se inicializa con una configuración aleatoria del sistema $\sigma$. Se inicia un contador $contador = 0$
- Se calcula la energía de estar en esa configuración y se guarda esa cantidad como H.
- Se elije una nueva configuración aleatoria $\sigma'$.
- Se calcula la energía de esta nueva configuración y se guarda como H'
- Se calcula la probabilidad p de pasar de $\sigma$ a $\sigma'$. 
- Se genera un número aleatorio $x$ entre 0 y 1. Si $x\le p$ Se re-nombra la configuración $\sigma = \sigma'$ y se incrementa en 1 el contador. Si $x>p$ sólo se incrementa en 1 el contador. 
- Si $contador < n$ con $n$ el número de pasos que se desea correr la simulación, entonces se regresa al paso 2. Sino, se termina el ciclo y se regresa la configuración $\sigma$. 

A veces muestrear todas las posibilidades de configuraciones es demasiado lento, pero se puede, en muchos casos, no cambiar toda la configuración $\sigma$, sino sólo un elemento. Digamos por ejemplo, que tenemos un conjunto de espines, unos apuntan hacia arriba, otros hacia abajo. En vez de generar configuraciones completas nuevas, se puede sólo cambiar uno de los espines. 

[9] Considera el siguiente Hamiltoniano de una cadena de espines: $$ H(\sigma) = -\mu \sum_i^N \sigma_i - J \sum_{\langle i,j\rangle} \sigma_i \cdot \sigma_j $$. Aquí $\mu$ y $J$ son constantes y la segunda suma se da sobre los vecinos $i$, o sea, si $i = j+1$ o $i = j-1$ se hace la suma, sino no. Y cada valor de $\sigma_i$ puede ser $\pm 1$. Haz una simulación de Metrópolis, para una cadena de 10 espines, $\mu = 1$ $J = 1$ y $KT$ varios valores. 

[10] Grafica el valor de $\sum_i \sigma_i$ como función del número de iteraciones. 

[11] Grafica el valor promedio entre las iteraciones 1000 y 10000 de $\sum_i \sigma_i$ como función de $KT$. 